# Integration of lakeFS with Dagster

## Versioning Information

In [ ]:
sourceBranch = "main"

## Import Python packages

In [ ]:
%xmode Minimal
import datetime
import os
import lakefs

## Set environment variables

In [ ]:
os.environ["LAKECTL_SERVER_ENDPOINT_URL"] = lakefsEndPoint
os.environ["LAKECTL_CREDENTIALS_ACCESS_KEY_ID"] = lakefsAccessKey
os.environ["LAKECTL_CREDENTIALS_SECRET_ACCESS_KEY"] = lakefsSecretKey

#### Verify lakeFS credentials by getting lakeFS version

In [ ]:
print("Verifying lakeFS credentials…")
try:
    v=lakefs.client.Client().version
except:
    print("🛑 failed to get lakeFS version")
else:
    print(f"…✅lakeFS credentials verified\n\nℹ️lakeFS version {v}")

## Create Repository

In [ ]:
try:
    repo=lakefs.repository(repo_name)
    print(f"Found existing repo {repo.id} using storage namespace {repo.properties.storage_namespace}")
except lakefs.exceptions.NotFoundException as f:
    print(f"Repository {repo_name} does not exist, so going to try and create it now.")
    try:
        repo=lakefs.repository(repo_name).create(storage_namespace=f"{storageNamespace}/{repo_name}")
        print(f"Created new repo {repo.id} using storage namespace {repo.properties.storage_namespace}")
    except lakefs.exceptions.LakeFSException as e:
        print(f"Error creating repo {repo_name}. Error is {e}")
except lakefs.exceptions.LakeFSException as e:
    print(f"Error getting repo {repo_name}: {e}")

## Start Dagit UI

In [ ]:
%%script bash --bg --out script_out --err script_error
dagit -h 0.0.0.0 -p 3000
echo "Dagit UI is running"

In [ ]:
if lakefsEndPoint.startswith('http://host.docker.internal'):
    lakefsUIEndPoint = lakefsEndPoint.replace('host.docker.internal','127.0.0.1')
elif lakefsEndPoint.startswith('http://lakefs'):
    lakefsUIEndPoint = 'http://127.0.0.1:28000'
else:
    lakefsUIEndPoint = lakefsEndPoint